## Annotation Data

In [1]:
from prodigy.components.db import connect

db = connect(db_id='mysql', db_settings= {"user": "",
                                            "password": "",
                                            "host": "cryptobot.c0qod4rtiyaw.eu-central-1.rds.amazonaws.com",
                                            "port": 3306,
                                            "database": "annotations"
                                        })

data = db.get_dataset("semanticsearch")

In [2]:
def checker(answer=None):
    if answer == 'accept':
        row['pos'].append(text)
    elif answer == 'reject':
        row['neg'].append(text)
    else:
        pass

In [3]:
import random

bag = []
corpus = []
row = {'query': '', 'pos': [], 'neg': []}

# Iterate all annotations from Prodigy and store all
# pos and neg in a bag corresponding to a query
for line in data:
    query = line['label']
    text = line['text']
    answer = line['answer']

    # for the first iteration
    if row['query'] == '':
        row['query'] = query
        checker(answer)

    # when there is a new query
    elif query != row['query']:
        bag.append(row)
        row = {'query': query, 'pos': [], 'neg': []}
        checker(answer)

    # append pos and negs to the same query
    else:
        checker(answer)

# for each query append only one pos and one neg
# normally there is more neg than pos, for that reason
# i decided to iterate over the negatives and append
# random pos (not the best solution)
for data in bag:
    negatives = data['neg']
    positives = data['pos']

    for i, value in enumerate(range(len(negatives))):
        try:
            row = {'query': data['query'],
                   'pos': random.choice(positives),
                   'neg': negatives[i]}

            corpus.append(row)
        except:
            pass

random.shuffle(corpus)
len(corpus)

108

In [4]:
corpus_train = corpus[:75]
corpus_test = corpus[75:]

## Training SBERT

In [5]:
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader, Dataset

model = SentenceTransformer('./msmarco/msmarco-distilbert-dot-v5')

### Using pytorch to load the data

In [6]:
class ProdigyDataset(Dataset):
    def __init__(self, corpus):
        self.corpus = corpus

    def __getitem__(self, item):
        query = self.corpus[item]

        query_text = query['query']
        pos_text = query['pos']
        neg_text = query['neg']

        return InputExample(texts=[query_text, pos_text, neg_text])

    def __len__(self):
        return len(self.corpus)

In [7]:
train_dataset = ProdigyDataset(corpus_train)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

#I use MultipleNegativesRankingLoss because I have a Positive passage and a hard Negative passage.
train_loss = losses.MultipleNegativesRankingLoss(model=model)

### Train

In [1]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=5,
          warmup_steps=1000,
          use_amp=True,
          checkpoint_path='./',
          checkpoint_save_steps=200,
          optimizer_params = {'lr': 2e-5}
          )

NameError: name 'model' is not defined

In [9]:
model.save('./saved_model')

## Evaluation

### MSMarco

In [13]:
from sentence_transformers.evaluation import TripletEvaluator

test_dataset = ProdigyDataset(corpus_test)
evaluator = TripletEvaluator.from_input_examples(test_dataset, batch_size=8)

model.evaluate(evaluator=evaluator)

0.65

### Finetuned

In [14]:
from sentence_transformers.evaluation import TripletEvaluator

model_fine = SentenceTransformer('./saved_model')
evaluator = TripletEvaluator.from_input_examples(test_dataset, batch_size=8)

model_fine.evaluate(evaluator=evaluator)

0.65